## sentiment scoring

In [2]:
import openai
import os
import pandas as pd
import time

openai.api_key = ""

In [3]:
data = pd.read_csv('sim_del_naver_news_data.csv', index_col=0)
data

,time,title,content,press,link,UTC-time,UTC-date
date,,,,,,,
2017-01-01,2017-01-01 07:00:00,한국 금융산업 위기… ‘우버 모멘트’ 를 넘어라,\n\n\n\n\n© News1저금리·저성장 전통 금융산업 위기4차 산업혁명...세...,뉴스1,https://n.news.naver.com/mnews/article/421/000...,2016-12-31 22:00:00,2016-12-31
2017-01-01,2017-01-01 07:02:03,"2017년 주식 투자…IT, 인프라 ‘솔깃’ 해외주식도 담아보자",\n\t\t\t[헤럴드경제=김지헌 기자] 올해도 지루한 박스피(박스권에 갇힌 코스피...,헤럴드경제,https://n.news.naver.com/mnews/article/016/000...,2016-12-31 22:02:03,2016-12-31
2017-01-01,2017-01-01 15:15:04,"英 파운드, 2016년 최악의 통화 ···'디지털통화' 비트코인 최고의 수익자산...",\n【뉴욕=정지원 특파원】 영국의 파운드가 2106년 최악의 통화로 기록됐다.\n ...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,2017-01-01 06:15:04,2017-01-01
2017-01-01,2017-01-01 16:03:04,"[신년기획]블록체인, 금융권 넘어 IT기업도 관심",\n\t\t\t“블록체인은 인터넷 이후 나온 가장 혁신적이며 파괴적인 기술이다.”블...,39면 1단,https://n.news.naver.com/mnews/article/030/000...,2017-01-01 07:03:04,2017-01-01
2017-01-01,2017-01-01 12:03:13,[2017 국제 핫이슈]블록체인 기술 확산,\n\t\t\t블록체인은 `제2의 인터넷` `넥스트 인터넷`으로 불린다. 새해에는 ...,전자신문,https://n.news.naver.com/mnews/article/030/000...,2017-01-01 03:03:13,2017-01-01
...,...,...,...,...,...,...,...
2023-08-30,2023-08-30 10:31:36,올해 상반기 금융사 '의심 거래' 보고 10% 늘어,\n\t\t\t올해 상반기 금융사들의 의심 거래 보고 건수가 지난해보다 10% 가까...,YTN,https://n.news.naver.com/mnews/article/052/000...,2023-08-30 01:31:36,2023-08-30
2023-08-30,2023-08-30 09:12:12,훨훨 난 비트코인…현물 ETF 상장 호재에 6% 급등,"\n비트코인, 박스권 깨고 3700만원 돌파\n\n\n\n[서울=뉴시스]이지영 기자...",뉴시스언론사 선정,https://n.news.naver.com/mnews/article/003/001...,2023-08-30 00:12:12,2023-08-30
2023-08-30,2023-08-30 09:41:23,"이원욱 ""이재명 1년, 미래도 유능함도 없는 민주당 돼""","\n이원욱 ""민주, 여전히 이재명의 대표리스크""""정치 훌리건의 공격 난무하는 친명 ...",뉴시스언론사 선정,https://n.news.naver.com/mnews/article/003/001...,2023-08-30 00:41:23,2023-08-30


In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [5]:
# def get_completion_4(prompt, model="gpt-4"):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0,
#     )
#     return response.choices[0].message["content"]

### prompt

In [6]:
prompt = "이전 지시사항은 모두 무시하십시오. 당신은 금융 전문가로 가정하며 금융자산 추천 경험이 있습니다. 다음 기사의 제목과 내용이 다음날 한국에서 거래되는 암호화폐의 가격에 좋은 소식이 확실하다면 1에 가까운 수를, 나쁜 소식이 확실하다면 –1에 가까운 수를, 불확실하다면 0에 가까운 수를 답하세요. 다시말해, 뉴스 기사를 읽고 다음달 암호화폐 가격에 대해 좋은 소식인지, 아니면 나쁜 소식인지를 –1과 1 사이의 실수로만 답하세요"

In [ ]:
data['score'] = (prompt + '제목:'+data['title']+ '내용:'+data['content']).apply(get_completion)
data

In [ ]:
data.to_csv('sentiment_result.csv', encoding="UTF-8-sig", index=False)